# DeepSeek-R1-Distill-Qwen-1.5B LoRA Fine-Tuning

## 1. Install Required Libraries

In [ ]:
!pip install transformers peft datasets accelerate pandas

## 2. Load and Preprocess Data

In [ ]:
import pandas as pd
from datasets import Dataset

# Load Excel file
df = pd.read_excel('data.xlsx')

# Extract text data
texts = df['text'].tolist()

# Convert to Hugging Face Dataset
dataset = Dataset.from_dict({'text': texts})

## 3. Load Model and Tokenizer

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

## 4. Configure LoRA

In [ ]:
from peft import LoraConfig, get_peft_model

# LoRA configuration
lora_config = LoraConfig(
    r=8,  # LoRA rank
    lora_alpha=32,  # LoRA alpha
    target_modules=["q_proj", "v_proj"],  # Target modules
    lora_dropout=0.1,  # Dropout
    bias="none",  # Bias
    task_type="CAUSAL_LM"  # Task type
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

## 5. Tokenize Dataset

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

## 6. Configure Training

In [ ]:
from transformers import TrainingArguments, Trainer

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=5e-5,
    fp16=True,  # Enable if GPU supports
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

## 7. Train the Model

In [ ]:
trainer.train()

## 8. Save the Fine-Tuned Model

In [ ]:
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")